### Import classes from autoqchem

In [3]:
from autoqchem.molecule import molecule
from autoqchem.sge_manager import sge_manager
from autoqchem.draw_utils import draw

### Set the level of logging
Autoqchem prints out log messages, while the logger has various verbosity levels that you can set. If you are running for the first time "INFO" is good, once you are comfortable you can switch it to "WARNING" or "ERROR".

In [4]:
import logging
logging.basicConfig(level=logging.INFO)

We will use a single smiles string for this test

In [30]:
smiles_str_list = ['CC', 'CCC']

### Initialize the molecule from smiles

In [31]:
mols = [molecule(s, num_conf=5) for s in smiles_str_list]

### Draw it!
It should look decent, OpenBabel performs MMFF94 optimization with a conformer search. If it doesn't look good, there might be issues with openbabel installation (this is rare, but can happen).

In [32]:
draw(mols[0].mol)

interactive(children=(Dropdown(description='confId', options=(0,), value=0), Output()), _dom_classes=('widget-…

<function autoqchem.draw_utils._graph_conf(m, confId=0, energies=[])>

### Test whether you can access SGE on Hoffman2 at UCLA
If you have an account on Hoffman2, change host to ```hoffman2.idre.ucla.edu```.

### Initialize the sge manager
change user parameter to your user

In [33]:
sm = sge_manager(user='wang10', host='hoffman2.idre.ucla.edu')

### Verify that you can create an ssh tunnel into the remote server
Follow prompts for password and duo authentication. This tunnel will be used to manage jobs from within the notebook

In [35]:
sm.connect()

INFO:autoqchem.sge_manager:Creating connection to hoffman2.idre.ucla.edu as wang10
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (password) successful!
INFO:autoqchem.sge_manager:Connected to hoffman2.idre.ucla.edu as wang10.


### Check that you can see your jobs on the remote server
If the following command returns a table with your jobs currently running on the server, it's a success. If you have no jobs running, only a header row will be displayed.

In [36]:
sm.qstat(summary=False)

'No jobs in queue'

In [43]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, light_basis_set='6-31G**', solvent='DMSO')

INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


In [44]:
sm.submit_jobs()

INFO:autoqchem.sge_manager:Submitting 1 jobs.
INFO:autoqchem.sge_manager:Submitted job e7e716719dcaf24de944ee06f7c5a24c, job_id: 7360980.


In [51]:
sm.qstat(summary=False)

'No jobs in queue'

In [52]:
sm.retrieve_jobs()

INFO:autoqchem.sge_manager:There are 0 running/pending jobs, 1 finished jobs.
INFO:autoqchem.sge_manager:Retrieving log files of finished jobs.
INFO:autoqchem.sge_manager:1 jobs finished successfully (all Gaussian steps finished normally). 0 jobs failed.


In [53]:
sm.upload_done_molecules_to_db(tags=["test_WCG"])

INFO:autoqchem.sge_manager:There are 1 finished molecules ['CC'].
INFO:autoqchem.sge_manager:Molecule CC has 0 / 1 duplicate conformers.
INFO:autoqchem.sge_manager:Removing 0 / 1 jobs and log files that contain duplicate conformers.
INFO:autoqchem.sge_manager:Uploaded descriptors to DB for smiles: CC, number of conformers: 1, DB molecule id 646801973834ff8df6a882de.
